In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
premise_file = "Resources/Covid-NA.csv"
premise_df = pd.read_csv(premise_file)
premise_df.head()

,"Country, Other",Total Cases,Total Deaths,Total Recovered,Active Cases,Serious / Critical Condition,Total Cases / 1M Population,Deaths / 1M Population,Total Tests,Tests / 1M Population,Population
0,USA,"42,634,054","688,486","32,347,726","9,597,842","25,209","127,897","2,065","615,684,393","1,846,976","333,347,223"
1,Mexico,"3,549,229","270,346","2,897,667","381,216","4,798","27,183","2,071","10,302,152","78,903","130,567,887"
2,Canada,"1,564,089","27,325","1,493,619","43,145",638,"41,007",716,"41,861,477","1,097,525","38,141,706"
3,Cuba,"776,125","6,601","730,673","38,851",458,"68,573",583,"8,835,501","780,641","11,318,258"
4,Guatemala,"521,093","12,907","468,216","39,970",5,"28,452",705,"2,373,843","129,612","18,315,046"


In [4]:
states_file = "Resources/states.csv"
states_df = pd.read_csv(states_file)
states_df.head()


,fips,country,state,county,level,lat,locationId,long,population,metrics.testPositivityRatio,...,actuals.vaccinesDistributed,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,lastUpdatedDate,url,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered,cdcTransmissionLevel
0,2,US,AK,NaN,state,NaN,iso1:us#iso2:us-ak,NaN,731545,0.104,...,1024575,411978,356784,9/18/2021,https://covidactnow.org/us/alaska-ak,0.563,0.488,1,763376,3
1,1,US,AL,NaN,state,NaN,iso1:us#iso2:us-al,NaN,4903185,0.175,...,6525240,2535635,2010919,9/18/2021,https://covidactnow.org/us/alabama-al,0.517,0.410,192,4383430,3
2,5,US,AR,NaN,state,NaN,iso1:us#iso2:us-ar,NaN,3017804,0.099,...,3925390,1656649,1336396,9/18/2021,https://covidactnow.org/us/arkansas-ar,0.549,0.443,18,2938812,3
3,4,US,AZ,NaN,state,NaN,iso1:us#iso2:us-az,NaN,7278717,0.110,...,9711660,4252340,3647534,9/18/2021,https://covidactnow.org/us/arizona-az,0.584,0.501,19,8045806,3
4,6,US,CA,NaN,state,NaN,iso1:us#iso2:us-ca,NaN,39512223,0.048,...,57390915,27860963,22769456,9/18/2021,https://covidactnow.org/us/california-ca,0.705,0.576,123,49458723,3
